# Denpendency

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/Colabs')

Mounted at /content/drive/


In [ ]:
import re
import csv
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from argparse import ArgumentParser
from sklearn import metrics

In [ ]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 

# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=97cc9f60b4248776195cffe5313df5a899f57ebd369a9e56b4340f2305455893
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/Colabs/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

text = "Đại học Quốc Gia Hà Nội"

word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Quốc_Gia', 'Hà_Nội']]


# Load and clean data shopee

In [ ]:
DATA_ROOT_DIR="/content/drive/MyDrive/Colabs/shopee-sentiment"
cols = ["id", "text", "text_ws", "label"]

In [ ]:
DATA_DIR = DATA_ROOT_DIR + '/train_preprocess.csv'
data = pd.read_csv(DATA_DIR, index_col=0)
data.head()

,text,class,preprocess_text
id,,,
dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy...",1,đến quán 2 lần thôi rất là thích quán tuy nằm ...
800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ...,0,đến quán vào tối chủ_nhật có band hát khá ổn t...
6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi g...,0,phục_vụ lâu quá mặc_dù khách rất vắng đợi gần ...
b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nê...",0,ko gian bé_tí quán chật_chội đông người nên ...
f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiế...",1,khi mình order đặt bánh thì nhận được sự tiếp_...


In [ ]:
dev_train_data, val_data, dev_train_label, val_label = train_test_split(data["preprocess_text"], data["class"], test_size=0.2)

# Model

In [ ]:
C_OPTIONS = np.logspace(-3, 3, 10)
C_OPTIONS

array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])

In [ ]:
def train(train_sens, y_train, tune_sgd=False):
    C_OPTIONS = np.logspace(-3, 3, 10)
    text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                         ('tfidf', TfidfTransformer(sublinear_tf=False)),
                         ('clf', LinearSVC(dual=False, tol=1e-3, penalty="l2", loss='squared_hinge'))])
    parameters = {
        'clf__C': C_OPTIONS,
    }

    score = 'f1'

    print("\nTuning parameters for Linear SVM\n")
    gs_clf = GridSearchCV(text_clf, parameters, cv=5, scoring=score,
                          n_jobs=1)
    gs_clf.fit(train_sens, y_train)
    for param_name in sorted(parameters.keys()):
        print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
    print("Best score (Grid search SVC): %f" % gs_clf.best_score_)

    model1 = LinearSVC(
        C=gs_clf.best_params_['clf__C'],
        loss='squared_hinge',
        penalty="l2",
        dual=False, tol=1e-3)

    if tune_sgd:

        print("\nTunning parameters for SGDClassifier\n")

        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                             ('tfidf', TfidfTransformer(sublinear_tf=False)),
                             ('clf', SGDClassifier(alpha=0.0001, n_jobs=1))])

        parameters = {
            'clf__l1_ratio': [0.01, 0.1, 0.15, 0.25, 0.3],
            'clf__loss': ['hinge', 'log', 'squared_hinge'],
            'clf__penalty': ['l1', 'l2', 'elasticnet'],
        }
        gs_clf = GridSearchCV(text_clf, parameters, cv=5, scoring=score,
                              n_jobs=-1)
        gs_clf.fit(train_sens, y_train)
        for param_name in sorted(parameters.keys()):
            print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
        print("Best score (Grid search SGD): %f" % gs_clf.best_score_)

        model2 = SGDClassifier(alpha=0.0001,
                             l1_ratio=gs_clf.best_params_['clf__l1_ratio'],
                             loss=gs_clf.best_params_['clf__loss'],
                             penalty=gs_clf.best_params_['clf__penalty'])
    else:
        model2 = SGDClassifier(alpha=0.0001, l1_ratio=0.1, loss='hinge', penalty="l2")

    print("\nEnsemble learning\n")

    model3 = RandomForestClassifier(n_estimators=200, max_depth=None,
                                  random_state=100)

    ensemble_clf = VotingClassifier(
        estimators=[('linearSVM', model1), ('sgd', model2),
                    ('rdf', model3),
                    ],
        voting='hard')

    ensembleModel = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                         ('tfidf', TfidfTransformer(sublinear_tf=False)),
                         ('clf', ensemble_clf),])

    ensembleModel.fit(train_sens, y_train)
    return ensembleModel

In [ ]:
def predict(ensembleModel, test_sens):
    return ensembleModel.predict(test_sens)

In [ ]:
# train_samples, train_labels = load_csv_data(DEV_TRAIN_CSV, textcol="text_ws")
# test_df = pd.read_csv(VAL_CSV)
# test_indices = [str(text) for text in test_df["id"]]
# test_samples = [str(text) for text in test_df["text_ws"]]
# test_labels = [intent for intent in test_df["label"]]

In [ ]:
train_samples = dev_train_data
train_labels = dev_train_label
test_samples = val_data
test_labels = val_label

In [ ]:
ensembleModel = train(train_samples, train_labels, tune_sgd=True)


Tuning parameters for Linear SVM

clf__C: 1000.0
Best score (Grid search SVC): 0.903143

Tunning parameters for SGDClassifier

clf__l1_ratio: 0.25
clf__loss: 'squared_hinge'
clf__penalty: 'l2'
Best score (Grid search SGD): 0.902024

Ensemble learning



In [ ]:
preds = predict(ensembleModel, test_samples)

In [ ]:
preds[0:20]

array([1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0])

In [ ]:
test_labels[0:20]

id
c1ba7154    1
03080cbb    1
7bff7b02    0
cde74902    1
54fe6333    1
daba90b8    0
536d8fae    1
973a2422    0
6155953c    1
3022c1ac    1
323aedc0    1
cf63328a    0
423f8a99    1
9b208da8    0
a572db78    1
2f9d6079    1
0a37ff88    0
507a7aec    0
d100a50c    1
49e77c38    1
Name: class, dtype: int64

In [ ]:
acc = metrics.f1_score(test_labels, preds)
print("Accuracy: {}".format(acc))
print(metrics.classification_report(test_labels, preds))

Accuracy: 0.9057413700590234
              precision    recall  f1-score   support

           0       0.92      0.87      0.90      2678
           1       0.88      0.93      0.91      2722

    accuracy                           0.90      5400
   macro avg       0.90      0.90      0.90      5400
weighted avg       0.90      0.90      0.90      5400



# Save and load model

In [ ]:
import torch

In [ ]:
SAVE_MODEL_PATH = "/content/drive/MyDrive/Colabs/saveModel/ensembleModelShopee.pt"
torch.save(ensembleModel, SAVE_MODEL_PATH)

In [ ]:
modelSave = torch.load(SAVE_MODEL_PATH)
modelSave

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 3))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 VotingClassifier(estimators=[('linearSVM',
                                               LinearSVC(C=1000.0, dual=False,
                                                         tol=0.001)),
                                              ('sgd',
                                               SGDClassifier(l1_ratio=0.25,
                                                             loss='squared_hinge')),
                                              ('rdf',
                                               RandomForestClassifier(n_estimators=200,
                                                                      random_state=100))]))])

# Classtify

In [ ]:
# test the model with some sentence
inputs = [
    "Món ăn ngon, đồ uống ngon, rẻ",
    "Món ăn ngon",
    "Tuy rẻ nhưng đồ ăn không ngon, đồ uống không ngon",
    "không gian chật hẹp, nhân viên nhiệt tình, đồ ăn ngon",
    "hàng rất tốt, đẹp, mẫu mã đa dạng, sẽ quay lại lần sau"
  ]
predictSen = predict(modelSave, inputs)
Labels = ["Negative", "Positive"]

for index ,item in enumerate(inputs):
    print(f"The sentence: '{inputs[index]}' has {Labels[predictSen[index]]} tone \n" )


The sentence: 'Món ăn ngon, đồ uống ngon, rẻ' has Positive tone 

The sentence: 'Món ăn ngon' has Positive tone 

The sentence: 'Tuy rẻ nhưng đồ ăn không ngon, đồ uống không ngon' has Negative tone 

The sentence: 'không gian chật hẹp, nhân viên nhiệt tình, đồ ăn ngon' has Negative tone 

The sentence: 'hàng rất tốt, đẹp, mẫu mã đa dạng, sẽ quay lại lần sau' has Positive tone 

